# 04 - Applied ML
## Question 1: Propensity score matching



In [1]:
#

## Question 2: Applied ML

### 1

### Loading The 20 newsgroups text dataset

In [2]:
from sklearn.datasets import fetch_20newsgroups
newsgroups_train = fetch_20newsgroups(subset='train')

### Compute TF-IDF features

In [3]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer()
vectors = vectorizer.fit_transform(newsgroups_train.data)

### Split train and test/val set

In [10]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(vectors, newsgroups_train.target, test_size=0.2, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(vectors, newsgroups_train.target, test_size=0.5, random_state=42)

print("Train set shape:", X_train.shape)
print("Test set shape:", X_test.shape)
print("Validation set shape:", X_val.shape)

Train set shape: (9051, 130107)
Test set shape: (5657, 130107)
Validation set shape: (5657, 130107)
